In [1]:
import os.path
import json
import importlib
import requests
import webbrowser
import base64
from datetime import datetime, timedelta
from requests.auth import HTTPBasicAuth

import util
from util import *

In [2]:
importlib.reload(util)
from util import *

## Set Up

Step by step instructions are at the following link, in case any of the steps below are unclear to you. Note that part of what you are learning here is how to figure these things out for yourself.

https://developer.spotify.com/documentation/web-api/quick-start/

0. Register for a Spotify account.  Ok for it to be a free account, not a premium one.

### Spotify Developer Steps

1. Go to https://developer.spotify.com/dashboard/login
2. On the Dashboard, use the `Create A Client ID` or, equivalently, click the `My New App` template box.
3.In Step 1 of 3
    - Name your app something like: `cs181-<login>`, using your Denison login.
    - Add whatever description you like.
    - Check `Desktop App` in answer to the question *"What are you building?*
    - Click `Next`
3. In Step 2 of 3, Select `Non-Commercial`
4. In Step 3 of 3, Click all three acknowledgement check boxes and Click `Submit`
    - You should see a message that your application has been created, and be on a page dedicated to that application.
5. Click the `Edit Settings` button and fill in the form as follows:
    - For `Website`, I use `https://personal.denison.edu/~bressoud/datasystems`.  You can use the same, or you can use some other web site that you are affiliated with.  For some API providers, they check to see if the web site exists.
    - For `Redirect URIs`, enter `https://caileighmarshall.github.io/cs181project/` (including the trailing slash), and click `Add`.
    - Click `Save` to save your application settings edits.
6. On your application page, click `Show Client Secret`, and leave the page up for the next set of steps.

### Record Credentials Information

1. In the same folder as this notebook, right-click on `creds.json` and `Open With` and select `Editor`.
2. Verify that, for the `"spotify"` entry, that the `"redirect_uri"` entry has the value `"https://caileighmarshall.github.io/cs181project/"`.
3. For the `"client_id"` entry, copy and paste the 32 character `Client ID` from your Application Dashboard as a string for that entry.
4. For the `"client_secret"` entry, copy and paste the 32 character `Client Secret` from your Application Dashboard as a string for that entry.
5. Save your changes to the `creds.json` file.  You may want to open it using the JSON navigator to make sure your JSON syntax is correct.

If the `read_creds()` function invocation in the cell below is not successful, check your steps again.

In [3]:
creds = read_creds("spotify", file="creds.json")

## Construct URL

The following three cells construct a URL.  An HTTP GET would be performed at this URL **by the Resource Owner**, taking them to a `Spotify` authentication and dialog, where they are asked if they approve of a particular application being able to perform certain categories of operations with their (the resource owner's) Spotify account.  The user can either approve, or can deny the application the ability to do these operations.

Run these three cells, and at the third one, you are effectively "Switching Roles" from being the application (in this notebook), and are transitioning to being the User/Resource Owner and interacting directly with the Provider, Spotify.

In [4]:
state = random_string()
creds['state'] = state
update_creds("spotify", creds, file="creds.json")

In [5]:
url = buildURL("/authorize", "accounts.spotify.com")
s = requests.Session()

scopes = ""
paramsD = {
    'client_id': creds['client_id'],
    'response_type': 'code',
    'redirect_uri': creds['redirect_uri'],
    'state': creds['state'],
    'show_dialog': 'true'
}
if scopes != "":
    paramsD['scope'] = scopes

In [6]:
req = requests.Request('GET', url, params=paramsD)

prepped = s.prepare_request(req)
user_url = prepped.url
print(user_url)

https://accounts.spotify.com/authorize?client_id=a01a0a6b1b1c45f2810cb36942393c57&response_type=code&redirect_uri=https%3A%2F%2Fcaileighmarshall.github.io%2Fcs181project%2F&state=2Z57BVUQ&show_dialog=true


> After you run the next cell, and you (presumably) approve (i.e. you give delegated authority to the application), you will be redirected to the `redirect_uri` site. When you get there, copy the long string of characters, called a `code` and displayed in red, and then come back to this notebook.

In [7]:
webbrowser.open(user_url)

True

## Communicate Code from User to App

For a regular application, the `redirect_uri` would take the resource owner to a web server under the control of the app, which would take the `code` provided, and put it in a database.

We simulate that with copy-and-paste.

Copy the code between the string delimiters, and then execute the following cell.

In [8]:
code = ""

Our Equivalent of conveying, through the user, the code generated by the provider and approved by the user, and storing it in our "database" of our credentials file.

In [9]:
creds['code'] = code
update_creds("spotify", creds, file="creds.json")

## Exchange Code for Token and Refresh Token

The `code` is not enough.  Although the app has the code, and it has been securely coveyed via an authenticated user, the application must securely authenticate and interact with the provider to obtain the `token`, which is then sufficient for API requests.

In [10]:
url = buildURL("/api/token", "accounts.spotify.com")
s = requests.Session()

formD = {
    'grant_type': 'authorization_code',
    'code': creds['code'],
    'redirect_uri': creds['redirect_uri'],
    'client_id': creds['client_id'],
    'client_secret': creds['client_secret']
}

In [11]:
req = requests.Request('POST', url, data=formD)

prepped = s.prepare_request(req)

In [12]:
prepped.url

'https://accounts.spotify.com/api/token'

In [13]:
prepped.body

'grant_type=authorization_code&code=AQCCviPArfcMGQcl-D-MywznFUZK-hkSR0GjaJOglVje11slhRgf85Lb04dUIFDbFIzQfUe4DslLunucJ8bsSDNzDDya19MtDH1YWDEEawU1XJhyhvwzD6FVj8kMgnViDAylVYjVhTXZhr25d7D_DY4Vvfsr6YOHywKTngu7oU_-qZfsqeAH9ha_Dyd417L2N6noUel6UuI0I6lUoPzaQc-oR7arWg&redirect_uri=https%3A%2F%2Fcaileighmarshall.github.io%2Fcs181project%2F&client_id=a01a0a6b1b1c45f2810cb36942393c57&client_secret=b77507cad3754f6fafca494510cc49e5'

In [14]:
response = s.send(prepped)
if response.status_code != 200:
    print("Error.  Status code:", response.status_code)
    print(response.text)

In [15]:
print(response.json())

{'access_token': 'BQDnzAnmLuxm2ClPVcrfhdeAG-QbsxSjjMZt3SrLbXF_KFAxohGkMHV3AJb9EVi1SO8Sak-1zP_1qTObSm9DUKIbs9zcUXGng9FD6lfyY-0Er_X5M_67Nnvev8M4569Eq_QdOtbzKMbzvhrMS2Fgvd1FrIgtrlP_Ww', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQDoXgv6zfJYGQkeYQenz9Va0h8xq8WUzU6ZZa43tTkqZGv5hD1H4k5jPAeRE9ut7x3DobHsk9RyTjpQxbPFF_VnpKu9BexW8AfsDkRIPbFYbV3ooQYntbCBg4MGeE4I1dU', 'scope': ''}


In [16]:
tokenD = response.json()

We then update our "database" with the `access_token` and the other information needed to make data requests:

In [17]:
creds['token'] = tokenD['access_token']
creds['refresh'] = tokenD['refresh_token']
creds['scope'] = tokenD['scope']
expiration = datetime.now() + timedelta(seconds=tokenD['expires_in'])
creds['expires'] = str(expiration)
update_creds("spotify", creds, file="creds.json")